# Mathematical derivations of the split-step Fourier method for solving Schrodinger's equation in both 1-D and 2-D 

Original author: Jake Vanderplas, email: vanderplas@astro.washington.edu. Adapted for 2-D: Han Chen, hansolo@vt.edu

This is a tutorial of the mathematical derivation necessary for transforming the time-dependent Schrodinger's equation (TDSE) from its original form into FFT-compatible (fast-fourier-transform-compatible) form that can be used to calculate the solution of TDSE.

The tutorial can be modified to solve heat equations by eliminating the potential term V.

Schrodinger's equation has the following form: ($j$ is the imaginary number, $j^2 = -1$)

$j\hbar \frac {\partial}{\partial t} \psi = -\frac{\hbar^2}{2m} \nabla ^2 \psi + V \psi$

in 1-D, the equation is:

1. $j\hbar \frac {\partial}{\partial t} \psi = -\frac{\hbar^2}{2m} \frac{\partial^2 }{\partial x^2} \psi + V \psi$

in 2-D, the equation is: (cartesian coordinates)

2. $j\hbar \frac {\partial}{\partial t} \psi = -\frac{\hbar^2}{2m} (\frac{\partial^2 }{\partial x^2} \psi+\frac{\partial^2 }{\partial y^2} \psi) + V \psi$

Apply Fourier transform properties on derivatives:

in 1-D: the transform independent variable pair x <--> k

3. $j\hbar \frac {\partial}{\partial t} \psi = \frac{\hbar^2 k^2}{2m} \widetilde{\psi} + V(j\frac{\partial}{\partial k})\widetilde{\psi}$

in 2-D: the transform independent variable pair x <--> u, y <-->v

4. $j\hbar \frac {\partial}{\partial t} \widetilde{\psi} = \frac{\hbar^2 u^2}{2m} \widetilde{\psi}+ \frac{\hbar^2 v^2}{2m} \widetilde{\psi} + V(-\frac{\partial}{\partial u} \frac{\partial}{\partial v}) \widetilde{\psi}$

Combine the previous two equations to approximate the solutions:

in 1-D:

5. $j\hbar \frac {\partial}{\partial t} \psi \simeq V(x,t) \psi$

6. $j\hbar\frac{\partial }{\partial t} \widetilde{\psi} \simeq \frac{\hbar^2 k^2}{2m} \widetilde{\psi} $

in 2-D:

7. $j\hbar \frac {\partial}{\partial t} \psi \simeq V(x,y,t) \psi$

8. $j\hbar\frac{\partial }{\partial t} \widetilde{\psi} \simeq \frac{\hbar^2 u^2}{2m} \widetilde{\psi}+ \frac{\hbar^2 v^2}{2m} \widetilde{\psi}$

Solving the above two sets of ordinary differential equations, and we arrive at the functions for small time-step evolution:

in 1-D:

9. $\psi(x, t + \Delta t) = \psi(x, t) \cdot exp(\frac {-j V(x) \Delta t}{\hbar})$

10. $\widetilde{\psi}(k, t + \Delta t) = \widetilde{\psi}(k, t) \cdot exp(\frac {-j \hbar k^2 \Delta t}{2m})$

in 2-D:

11. $\psi(x, y, t + \Delta t) = \psi(x, y, t) \cdot exp(\frac {-j V(x,y) \Delta t}{\hbar})$

12. $\widetilde{\psi}(u, v, t + \Delta t) = \widetilde{\psi}(u, v, t) \cdot exp(\frac {-j \hbar (u^2 + v^2) \Delta t}{2m})$

Equation 9 to 12 will come in handy later.

Now we try to figure out how to transform between $\psi$ and $\widetilde{\psi}$.
We start with their very definitions. Eventually, we want to implement FFT:

in 1-D:

13. $\widetilde{\psi}(k, t) = \frac{1}{\sqrt{2\pi}} \int_{-\infty}^{\infty} \psi(x, t) e^{-j k x} dx$

in 2-D:

14. $\widetilde{\psi}(u,v, t) = \frac{1}{2\pi} \int_{-\infty}^{\infty} \int_{-\infty}^{\infty} \psi(x, y, t) e^{-j(u x + v y)} dx dy$

As an approximation, we switch infinites to a finite domain:

in 1-D:

15. $\widetilde{\psi}(k, t) = \frac{1}{\sqrt{2\pi}} \int_{a}^{b} \psi(x, t) e^{-ikx} dx$

in 2-D:

16. $\widetilde{\psi}(u,v, t) = \frac{1}{2\pi} \int_{c}^{d} \int_{a}^{b} \psi(x, y, t) e^{-i(u x + v y)} dx dy$

where a, b, c, and d would be the domain boundaries that you decide on

Now we transform equation 15, and 16 into the form of a discrete sum:

in 1-D:

17. $\widetilde{\psi}(k_m, t) \simeq \frac{\Delta x}{\sqrt{2\pi}} \sum_{n=0}^{N-1} \psi(x_n, t) e^{-ikx_n} $

where:

* $x_n = x_0 + n \Delta x, n = 0,1,...,N, x_0 = a$, and,  $x_0 + N \Delta x = b$

* $k_n = k_0 + m \Delta k, m = 0,1,...,M, k_0 = - \frac{\pi}{\Delta x}$, and, $\Delta k = \frac {2 \pi}{N \Delta x}$

in 2-D:

18. $\widetilde{\psi}(u_r, v_s, t) \simeq \frac{\Delta x \Delta y}{2\pi} \sum_{m=0}^{M-1} \sum_{n=0}^{N-1} \psi(x_n, y_m, t) e^{-i (u_r x_n + v_s y_m)} $

where:

* $x_n = x_0 + n \Delta x, n = 0,1,...,N, x_0 = a$, and,  $x_0 + N \Delta x = b$

* $y_m = y_0 + m \Delta y, m = 0,1,...,M, y_0 = c$, and,  $y_0 + M \Delta y = d$

* $u_r = u_0 + r \Delta u, r = 0,1,...,R, u_0 = - \frac{\pi}{\Delta x}$, and, $\Delta u = \frac {2 \pi}{N \Delta x}$

* $v_s = v_0 + s \Delta v, s = 0,1,...,S, v_0 = - \frac{\pi}{\Delta y}$, and, $\Delta v = \frac {2 \pi}{M \Delta y}$

To proceed further, substitute $k_n = k_0 + m \Delta k$ into equation 17, and  $u_r = u_0 + r \Delta u$, $v_s = v_0 + s \Delta v$ into equation 18:

in 1-D:

19. $\widetilde{\psi}(k_m, t) = \frac{\Delta x}{\sqrt{2\pi}} \sum_{n=0}^{N-1} \psi(x_n, t) e^{-j (k_0 + m \Delta k) (x_0 + n \Delta x)}$

in 2-D:

20. $\widetilde{\psi}(u_r, v_s, t) = \frac{\Delta x \Delta y}{2\pi} \sum_{m=0}^{M-1} \sum_{n=0}^{N-1} \psi(x_n, y_m, t) e^{-j (u_0 + r \Delta u) (x_0 + n \Delta x) - j (v_0 + s \Delta v) (y_0 + m \Delta y)}$

Equation 19 and 20 can be simplified into the following forms: (the lengthy derivations [can be seen here](https://raw.githubusercontent.com/The-real-Han-Chen/Single-slit-and-double-slits-diffraction-Schrodinger-s-equation-in-2D-/master/derivations_and_demos/derivation.part2.jpg))

in 1-D:

21. $\widetilde{\psi}(k_m, t) = \frac{\Delta x}{\sqrt{2\pi}} \sum_{n=0}^{N-1} \psi(x_n, t) e^{-j m \Delta k x_0} e^{-j k_0 x_n} e^{-2 \pi j m n / N}$

in 2-D:

22. $\widetilde{\psi}(u_r, v_s, t) = \frac{\Delta x \Delta y}{2\pi} \sum_{m=0}^{M-1} \sum_{n=0}^{N-1} \psi(x_n, y_m, t) e^{-j r \Delta u x_0 - j s \Delta v y_0} e^{-j u_0 x_n - j v_0 y_m} e^{-2 \pi j r n/N - 2 \pi j s m/M}$

We can shift the constant exponential term out of the sum to the left side:

in 1-D:

21. $\widetilde{\psi}(k_m, t) e^{j m \Delta k x_0} = \frac{\Delta x}{\sqrt{2\pi}} \sum_{n=0}^{N-1} \psi(x_n, t) e^{-j k_0 x_n} e^{-2 \pi j m n / N}$

in 2-D:

22. $\widetilde{\psi}(u_r, v_s, t) e^{j r \Delta u x_0 + j s \Delta v y_0}  = \frac{\Delta x \Delta y}{2\pi} \sum_{m=0}^{M-1} \sum_{n=0}^{N-1} \psi(x_n, y_m, t) e^{-j u_0 x_n - j v_0 y_m} e^{-2 \pi j r n/N} e^{- 2 \pi j s m/M}$

If we introduce the following definitions:

in 1-D:

23. $\widetilde{\psi}_{mod,k} = \widetilde{\psi}(k_m, t) e^{j m \Delta k x_0}$

24. $\psi_{mod,x} = \psi(x_n, t) e^{-j k_0 x_n}$

in 2-D:

25. $\widetilde{\psi}_{mod,uv} = \widetilde{\psi}(u_r, v_s, t) e^{j r \Delta u x_0 + j s \Delta v y_0}$

26. $\psi_{mod,xy} = \psi(x_n, y_m, t) e^{-j u_0 x_n - j v_0 y_m}$

Then equation 21 and 22 becomes:

in 1-D:

27. $\widetilde{\psi}_{mod,k} = \frac{\Delta x}{\sqrt{2\pi}} \sum_{n=0}^{N-1} \psi_{mod,x} e^{-2 \pi j m n / N}$

in 2-D:

28. $\widetilde{\psi}_{mod,uv} = \frac{\Delta x \Delta y}{2\pi} \sum_{m=0}^{M-1} \sum_{n=0}^{N-1} \psi_{mod,xy} e^{-2 \pi j r n/N} e^{- 2 \pi j s m/M}$

Equation 27 and 28 fit the definition of function ```fft``` and ```fft2``` from ```scipy.fft```, and can be turned into:

in 1-D:

29. $\widetilde{\psi}_{mod,k} = \frac{\Delta x}{\sqrt{2\pi}}$ ```fft```$(\psi_{mod,x})$

in 2-D:

30. $\widetilde{\psi}_{mod,uv} = \frac{\Delta x \Delta y}{2\pi}$ ```fft2```$(\psi_{mod,xy})$

The definitions of $\widetilde{\psi}_{mod,k}$, $(\psi_{mod,x})$, $\widetilde{\psi}_{mod,uv}$, and $(\psi_{mod,xy})$ can be found in equation 23 to 26

```ifft``` and ```ifft2``` can also be used:

in 1-D:

31. ```ifft```$(\widetilde{\psi}_{mod,k}) = \frac{\Delta x}{\sqrt{2\pi}} \psi_{mod,x}$

in 2-D:

32. ```ifft2```$(\widetilde{\psi}_{mod,uv}) = \frac{\Delta x \Delta y}{2\pi} \psi_{mod,xy}$

Finally, to reach the solution at the desired time step, [the split-step loop](https://raw.githubusercontent.com/The-real-Han-Chen/Single-slit-and-double-slits-diffraction-Schrodinger-s-equation-in-2D-/master/derivations_and_demos/derivation.part3.jpg) is used.